In [ ]:
from lib.imports import *
from torch.utils.data import TensorDataset,DataLoader

device = 'cuda'
data_dir = 'data/andrew-data'

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(1500,64)
        self.do1 = nn.Dropout(p=.2)
        self.fc2 = nn.Linear(64,1)
    
    def forward(self, x):
        x = self.fc1(x)
        x = relu(x)
        # x = self.do1(x)
        x = self.fc2(x)

        return x  
model = MLP().to(device=device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=3e-4)

In [ ]:
X_train = torch.load('X_train.pt')
y_train = torch.load('y_train.pt')
X_test = torch.load('X_test.pt')
y_test = torch.load('y_test.pt')

trainloader = DataLoader(TensorDataset(X_train,y_train),batch_size=64,shuffle=True)
devloader = DataLoader(TensorDataset(X_test,y_test),batch_size=64,shuffle=True)

In [ ]:
loss_tr = []
loss_dev = []

In [ ]:
model.load_state_dict(torch.load(f'model_0.184.pt'))

In [ ]:
lre = torch.linspace(-3,-1,len(trainloader))
lrs = 10**lre
lri = []
lossi = []
model.train()
loss_tr_total = 0
for i,(X_tr,y_tr) in tqdm(enumerate(trainloader)):
    optimizer = torch.optim.Adam(model.parameters(),lr=lrs[i])
    X_tr,y_tr = X_tr.to(device),y_tr.to(device)
    logits = model(X_tr)
    loss = criterion(logits,y_tr)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss_tr_total += loss.item()
    lri.append(lre[i])
    lossi.append(loss.item())
plt.plot(lri,lossi)

In [ ]:
# optimizer = torch.optim.Adam(model.parameters(),lr=1e-4)
for i in tqdm(range(5)):
    model.train()
    loss_tr_total = 0
    for (X_tr,y_tr) in trainloader:
        X_tr,y_tr = X_tr.to(device),y_tr.to(device)
        logits = model(X_tr)
        loss = criterion(logits,y_tr)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_tr_total += loss.item()
    loss_tr.append(loss_tr_total/len(trainloader))
    model.eval()
    loss_dev_total = 0
    for (X_dv,y_dv) in devloader:
        X_dv,y_dv = X_dv.to(device),y_dv.to(device)
        logits = model(X_dv)
        loss = criterion(logits,y_dv)
        loss_dev_total += loss.item()
    loss_dev.append(loss_dev_total/len(devloader))
    plt.plot(loss_tr[-30:])
    plt.plot(loss_dev[-30:])
    plt.savefig('loss.jpg')
    plt.close()
    print(f'Epoch {i} Train: {loss_tr_total/len(trainloader)} Dev: {loss_dev_total/len(devloader)}')
    torch.save(model.state_dict(),f'model_{loss_dev[-1]:.3f}.pt')

: 

In [ ]:
model.load_state_dict(torch.load(f'model_0.184.pt'))

In [ ]:
from lib.utils import cms
# # test confusion matrices
y_true = torch.Tensor()
y_pred = torch.Tensor().cuda()
for (X,y) in tqdm(devloader):
    y_true = torch.cat([y_true,y.round()])
    y_pred = torch.cat([y_pred,sigmoid(model(X.cuda())).round()])
y_pred = y_pred
cms(y_true=y_true,y_pred=y_pred.detach().cpu(),current_date=None)

In [ ]:
model = MLP().cuda()
model.load_state_dict(torch.load('model-epoch9.pt'))
logits = model(X.cuda())
y_pred = nn.Sigmoid()(logits)
y_pred = y_pred.detach().cpu()
y_pred = y_pred.round()
y_pred = torch.cat([torch.zeros(50,1),y_pred,torch.zeros(49,1)]).long()
df['y_pred'] = y_pred*10